In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [122]:
# Laste poll-of-polls data
df = pd.read_csv("../data/all_polls.csv", sep=",", encoding='ISO-8859-1')

In [123]:
# Convert to datetime and set the date to the end of the month
df["Dato"] = pd.to_datetime(df["Dato"])

In [124]:
# Sort values and set index
df = df.sort_values("Dato")
df.set_index("Dato", inplace=True)

In [125]:
df_en=df[["Ap","Høyre","Frp","SV","Sp","KrF","Venstre","MDG","Rødt"]]

In [126]:
df_en

,Ap,Høyre,Frp,SV,Sp,KrF,Venstre,MDG,Rødt
Dato,,,,,,,,,
2008-01-09,32.7,14.1,23.2,7.1,6.4,7.8,6.0,0.0,0.0
2008-01-10,30.7,16.2,23.4,8.1,5.4,5.6,7.4,0.0,0.0
2008-01-18,29.5,17.7,25.6,6.4,5.0,6.4,6.1,0.0,0.0
2008-01-21,28.4,20.6,24.7,5.6,6.1,6.7,5.2,0.0,1.6
2008-01-21,30.7,18.3,24.2,7.1,5.4,6.2,5.4,0.0,1.1
...,...,...,...,...,...,...,...,...,...
2025-09-05,27.7,15.0,21.0,5.8,5.8,4.2,3.9,6.4,5.3
2025-09-05,26.4,14.3,20.4,6.0,6.2,4.2,4.3,7.2,6.9
2025-09-06,27.0,13.5,20.8,5.3,7.0,5.8,4.1,6.0,5.6


In [127]:
n_timesteps = 5   # past steps to look at
n_future = 12      # steps into the future we want to predict
n_features = df_en.shape[1]  
series_names = df_en.columns

In [128]:
n_features

9

In [129]:
n_timesteps = 20  # number of past days to look at
X, y = [], []

for i in range(len(df_en) - n_timesteps):
    X.append(df_en.values[i:i+n_timesteps, :])  # last 5 days of all 9 series
    y.append(df_en.values[i+n_timesteps, :])    # predict all 9 series next day

X = np.array(X)  # shape: (num_samples, n_timesteps, 9)
y = np.array(y)  # shape: (num_samples, 9)

In [130]:
split = int(len(X) * 0.97)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [131]:
X_train

array([[[32.7, 14.1, 23.2, ...,  6. ,  0. ,  0. ],
        [30.7, 16.2, 23.4, ...,  7.4,  0. ,  0. ],
        [29.5, 17.7, 25.6, ...,  6.1,  0. ,  0. ],
        ...,
        [28.5, 18. , 25.2, ...,  6.4,  0. ,  0. ],
        [28.7, 19.1, 26.3, ...,  5.5,  0. ,  1.6],
        [29.6, 17.4, 26.5, ...,  6.7,  0. ,  1.5]],

       [[30.7, 16.2, 23.4, ...,  7.4,  0. ,  0. ],
        [29.5, 17.7, 25.6, ...,  6.1,  0. ,  0. ],
        [28.4, 20.6, 24.7, ...,  5.2,  0. ,  1.6],
        ...,
        [28.7, 19.1, 26.3, ...,  5.5,  0. ,  1.6],
        [29.6, 17.4, 26.5, ...,  6.7,  0. ,  1.5],
        [30.6, 18.5, 22. , ...,  6.4,  0. ,  1. ]],

       [[29.5, 17.7, 25.6, ...,  6.1,  0. ,  0. ],
        [28.4, 20.6, 24.7, ...,  5.2,  0. ,  1.6],
        [30.7, 18.3, 24.2, ...,  5.4,  0. ,  1.1],
        ...,
        [29.6, 17.4, 26.5, ...,  6.7,  0. ,  1.5],
        [30.6, 18.5, 22. , ...,  6.4,  0. ,  1. ],
        [27.1, 17.2, 27.1, ...,  5.9,  0. ,  3.1]],

       ...,

       [[18.9, 20.3, 23.

In [132]:
len(X_test)

56

In [133]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(n_timesteps, 9)),
    LSTM(64),
    Dense(9)
])

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [134]:
optimizer = Adam(learning_rate=0.001)  # default 0.001
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=8)

Epoch 1/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 74.8506 - val_loss: 17.3210
Epoch 2/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 18.5063 - val_loss: 12.5185
Epoch 3/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 13.3822 - val_loss: 13.3530
Epoch 4/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 13.0448 - val_loss: 13.4016
Epoch 5/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 11.7226 - val_loss: 13.6220
Epoch 6/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 9.6916 - val_loss: 14.4684
Epoch 7/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 8.5085 - val_loss: 12.6546
Epoch 8/100
224/224 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 6.8106 - val_loss: 10.8191
Epoch 9/100
196/224 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 5.5384

In [ ]:
y_pred = model.predict(X_test)  # shape: (num_test, 9)

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i, col in enumerate(df_en.columns):
    plt.subplot(3, 3, i+1)
    plt.plot(y_test[:, i], label='Actual')
    plt.plot(y_pred[:, i], label='Predicted')
    plt.title(col)
    plt.xlabel('Time step')
    plt.ylabel('Value')
    plt.legend()
plt.tight_layout()
plt.show()
